## One Line Solution

Now we have learned some basic concepts of DataFrame. You may wondering whether we can simplify the code furthermore. In the **Split-Apply-Combine** chapter, we solve the problem, *What's the most common complaint type?*, with the following code:

In [3]:
using DataFrames
complaints = readtable("./data/311-service-requests.csv")
complaints_by_type = by(complaints, :Complaint_Type, nrow)
sort!(complaints_by_type, cols=:x1, rev=true)
complaints_by_type[1:10, :]

,Complaint_Type,x1
1,HEATING,14200
2,GENERAL CONSTRUCTION,7471
3,Street Light Condition,7117
4,DOF Literature Request,5797
5,PLUMBING,5373
6,PAINT - PLASTER,5149
7,Blocked Driveway,4590
8,NONCONST,3998
9,Street Condition,3473
10,Illegal Parking,3343


It's very verbose here. We can use the `|>` operator to chain them together:

In [11]:
readtable("./data/311-service-requests.csv") |> 
    x -> by(x, :Complaint_Type, nrow) |>
    x -> sort(x, cols=:x1, rev=true) |> 
    x -> x[1:10, :]

,Complaint_Type,x1
1,HEATING,14200
2,GENERAL CONSTRUCTION,7471
3,Street Light Condition,7117
4,DOF Literature Request,5797
5,PLUMBING,5373
6,PAINT - PLASTER,5149
7,Blocked Driveway,4590
8,NONCONST,3998
9,Street Condition,3473
10,Illegal Parking,3343


Although it is better now, the lambda functions are very annoying. I really miss the `->` macro in Clojure. Thankfully, there's a library named [lazy](https://github.com/MikeInnes/Lazy.jl), which provides many userful macros. The `@>`macro is very similar to `->` in Clojure.

In [27]:
using Lazy

@> readtable("./data/311-service-requests.csv") begin
    by(:Complaint_Type, nrow)
    sort(cols=:x1, rev=true)
    view(1:10)
end

,Complaint_Type,x1
1,HEATING,14200
2,GENERAL CONSTRUCTION,7471
3,Street Light Condition,7117
4,DOF Literature Request,5797
5,PLUMBING,5373
6,PAINT - PLASTER,5149
7,Blocked Driveway,4590
8,NONCONST,3998
9,Street Condition,3473
10,Illegal Parking,3343


Much better now! 

Since the *select-split-combine* operations are so common in daily programming, we can use the LINQ style macros provided by the [Query](http://www.david-anthoff.com/Query.jl/stable/index.html) library.

In [29]:
using Query

In [64]:
res = readtable("./data/311-service-requests.csv") |>
    @query(i, begin
        @group i by i.Complaint_Type into g
        @select {Complaint_Type=g.key, Count=length(g)}
        end) |>
    @query(j, begin
        @orderby descending(j.Count)
        @select j
        @collect DataFrame
        end)

res[1:10, :]

,Complaint_Type,Count
1,HEATING,14200
2,GENERAL CONSTRUCTION,7471
3,Street Light Condition,7117
4,DOF Literature Request,5797
5,PLUMBING,5373
6,PAINT - PLASTER,5149
7,Blocked Driveway,4590
8,NONCONST,3998
9,Street Condition,3473
10,Illegal Parking,3343


Ah, looks more complicate now... Maybe the syntax can be simplified later. You can explore some [Experimental Features](http://www.david-anthoff.com/Query.jl/stable/experimental.html) for how to make it more consise.

Personaly, I like the more functional style~